In [14]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np

from numpy import array
from sklearn.decomposition import PCA

In [15]:
# A class to initialize the super pixels, of the form - [h,w,l,a,b].
class SuperPixels(object):

    def __init__(self, h, w, l=0, a=0, b=0):
        self.update(h, w, l, a, b)
        self.pixels = [] # tracks associated input pixels

    def update(self, h, w, l, a, b):
        self.h = h
        self.w = w
        self.l = l
        self.a = a
        self.b = b

In [16]:
# returns an object of class SuperPixel
def make_superPixel(h, w,img):
    return SuperPixels(h, w,img[h,w][0],img[h,w][1],img[h,w][2])

In [17]:
# calculate gradient;  G(x,y) = ‖I(x+ 1,y)−I(x−1,y)‖²+‖I(x,y+ 1)−I(x,y−1)‖²
def calc_gradient(h, w,img,img_w,img_h):
    
    l = pow(img[h, w+1][0] - img[h, w - 1][0] , 2) #
    a = pow(img[h, w+1][1] - img[h, w - 1][1] , 2)
    b = pow(img[h, w+1][2] - img[h, w - 1][2] , 2)
    x_diff = math.sqrt(l + a + b)
    
    l = pow(img[h + 1, w][0] - img[h - 1, w][0] , 2)
    a = pow(img[h + 1, w][1] - img[h - 1, w][1] , 2)
    b = pow(img[h + 1, w][2] - img[h - 1, w][2] , 2)
    y_diff = math.sqrt(l + a + b)
    
    grad = x_diff + y_diff
    return grad

In [18]:
# reassign clusters based on gradient
def perturb_center(clusters, img):
    for c in clusters:
        center_gradient = calc_gradient(c.h, c.w,img,img_w,img_h)
        w = c.w
        h = c.h
        for x in range(-1, 2):
            for y in range(-1, 2):
                W = w + x
                H = h + y
            
                if H == img_h or W == img_w: # might need to increase to H-1,W-1
                    new_gradient == math.inf
                else:
                    new_gradient = calc_gradient(H,W, img,img_w,img_h)
                    
                if new_gradient < center_gradient: # reassign to lower gradient position
                    c.update(H, W, img[H,W][0], img[H,W][1],img[H,W][2])
                    center_gradient = new_gradient # keeps track of lowest found in neighborhood so far

In [19]:
# Defines initial cluster centers distanced at S
def initial_cluster_center(S,img,img_h,img_w):
    
    clusters = []
    h = S // 2 
    w = S // 2
    while h < img_h:
        while w < img_w:
            clusters.append(make_superPixel(h, w,img))
            w += S
        w = S // 2
        h += S
        
    return clusters

In [20]:
# replaces cluster center's (x,y) position w/ mean of the input pixels 
def update_cluster_mean(clusters, image):
    for c in clusters:
        sum_h = sum_w = number = 0
        sum_l, sum_a, sum_b = 0,0,0
        n = len(c.pixels)
        
        for p in c.pixels:
            sum_h += p[0]
            sum_w += p[1]            
            sum_l += image[p[0],p[1]][0]
            sum_a += image[p[0],p[1]][1]
            sum_b += image[p[0],p[1]][2]
            
        H = sum_h // n
        W = sum_w // n
        avg_l = sum_l / n # don't use floor division for LAB values
        avg_a = sum_a / n
        avg_b = sum_b / n
        c.update(H, W, avg_l, avg_a, avg_b)

In [21]:
# converts LAB images back to RGB and save it
def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    io.imsave(path, rgb_arr)

In [22]:
# replace the color of each pixel in a cluster by the color of the cluster's center
def avg_color_cluster(img,name,clusters):
    image = np.copy(img)
        
    for c in clusters:
        for p in c.pixels:
            image[p[0],p[1]][0] = c.l
            image[p[0],p[1]][1] = c.a
            image[p[0],p[1]][2] = c.b
        # To change the color of cluster center to Black
        image[c.h, c.w][0] = 0
        image[c.h, c.w][1] = 0
        image[c.h, c.w][2] = 0        
        
    lab2rgb(name, image)

In [23]:
# assign pixels to center w/ best color & position proximity in a 2S x 2S neighborhood
def assign_pixels(clusters, S, img, img_h, img_w):
    
    prev = {} # tracks clusters found before best cluster identified
    
    # initialize the distance between pixels and cluster center as infinity
    dis = np.full((img_h, img_w), np.inf) 
    
    for c in clusters:
        for h in range(c.h - 2 * S, c.h + 2 * S):       
            if h < 0 or h >= img_h: continue           # check image height boundaries
            for w in range(c.w - 2 * S, c.w + 2 * S):  # 2S x 2S neighborhood
                if w < 0 or w >= img_w: continue       # check image width boundaries
                l, a, b = img[h,w]                     # load input pixel
                Dc = math.sqrt(math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2))
                Dp = math.sqrt(math.pow(h - c.h, 2) + math.pow(w - c.w, 2))
                m = 20 # controls compactness
                D = Dc + (Dp * m / S) # don't use floor division
                # check if closest center found so far
                if D < dis[h,w]:
                    if (h, w) in prev:
                        prev[(h, w)].pixels.remove((h, w)) # removes an input pixel prev. associated w/ another cluster
                        prev[(h, w)] = c # re-assigned cluster
                        c.pixels.append((h, w))
                    else:
                        prev[(h, w)] = c
                        c.pixels.append((h, w))
                    dis[h, w] = D 

In [24]:
# function for the Simple Linear Iterative Clustering
def slic(S,img,img_h,img_w):
        
    clusters = initial_cluster_center(S,img,img_h,img_w) # initialize superpixel clusters as a grid 
#     perturb_center(clusters, img)
    
    for i in range(10): # usually converges after 10 iterations
        assign_pixels(clusters,S,img,img_h,img_w)
        update_cluster_mean(clusters,img)
    name = 'slic_m{m}_k{k}.png'.format(m=m, k=k)
    print("slic ran")
    avg_color_cluster(img, name, clusters)
    return clusters

In [25]:
# read the input RGB image
rgb = io.imread("city-small.jpg",plugin='matplotlib')
print(rgb.shape)

# convert RGB to LAB
img = color.rgb2lab(rgb)

k = 50   # Number of Super pixels, # adjust later
m = 20    # Constant for normalizing the color proximity, range of m = [1,40], # paper uses '45'

img_h = img.shape[0] # Image Height
img_w = img.shape[1] # Image Width

N = img_h * img_w  # Total number of pixels in the image
S = int(math.sqrt(N / k)) # initial length of one superpixel along one dimension


(364, 600, 3)


In [ ]:
cluster = slic(S,img,img_h,img_w)